# Making pipelines 
Dataset A

In [12]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt

#import custom functions
import Configure_Dataset
import Impute
import Normalize
import Combine
import Transform
import copy

In [2]:
data, y1, y2 = Configure_Dataset.configure('Competition1_raw_data.xlsx')

In [3]:
data.head()

,C1,C2,C4,C7,%_long_sentences,%_long_words,%_positive_words,%_negative_words,%_uncertain_words,C3',C5',C6',one_hot_Manufacturing,one_hot_Other,one_hot_Public Services
0,122.0,1.0,0.029074,51.345,0.640426,0.054250,0.004875,0.009199,0.010929,1,3.864345,11.111111,1,0,0
1,259.0,0.0,-0.013352,25.936,0.644753,0.051395,0.003258,0.011105,0.010876,0,12.028832,0.000000,1,0,0
2,90.0,1.0,0.020715,7.378,0.636816,0.061764,0.011593,0.006271,0.011403,0,3.369134,0.000000,1,0,0
3,209.0,1.0,0.020023,8.526,0.539634,0.061630,0.009686,0.007144,0.013319,0,3.299697,0.000000,1,0,0
4,80.0,1.0,-0.034895,632.298,0.587413,0.048550,0.004518,0.010047,0.011261,1,3.726269,5.000000,0,0,1


In [4]:
def apply_day_bins(col):
    #bin the days column into equal height bins
    
    q1 = data['C1'].describe()['25%']
    q2 = data['C1'].describe()['50%']
    q3 = data['C1'].describe()['75%']
    
    
    if col <= q1:
        return(1)
    if col > q1 and col <=q2: 
        return(2)
    if col > q2 and col <= q3:
        return(3)
    else:
        return(4)
    
  

In [5]:
data['C1'] = data['C1'].apply(apply_day_bins)


In [6]:
data.head(10)

,C1,C2,C4,C7,%_long_sentences,%_long_words,%_positive_words,%_negative_words,%_uncertain_words,C3',C5',C6',one_hot_Manufacturing,one_hot_Other,one_hot_Public Services
0,3,1.0,0.029074,51.345,0.640426,0.054250,0.004875,0.009199,0.010929,1,3.864345,11.111111,1,0,0
1,4,0.0,-0.013352,25.936,0.644753,0.051395,0.003258,0.011105,0.010876,0,12.028832,0.000000,1,0,0
2,2,1.0,0.020715,7.378,0.636816,0.061764,0.011593,0.006271,0.011403,0,3.369134,0.000000,1,0,0
3,4,1.0,0.020023,8.526,0.539634,0.061630,0.009686,0.007144,0.013319,0,3.299697,0.000000,1,0,0
4,1,1.0,-0.034895,632.298,0.587413,0.048550,0.004518,0.010047,0.011261,1,3.726269,5.000000,0,0,1
5,2,1.0,-0.057556,197.591,0.643137,0.045280,0.003445,0.005334,0.010056,1,5.724394,0.000000,0,0,1
6,3,1.0,0.004106,5.146,0.765657,0.044295,0.005204,0.006727,0.008567,0,3.463095,0.000000,1,0,0
7,2,0.0,0.040620,NaN,0.826347,NaN,0.004899,0.010689,0.010868,0,2.421469,0.000000,1,0,0
8,2,1.0,-0.029316,279.600,0.540323,0.061905,0.003915,0.008371,0.015797,0,4.816591,0.000000,0,0,1
9,2,1.0,-0.024925,494.008,0.514286,0.050252,0.008868,0.009853,0.016094,1,3.056608,0.000000,1,0,0


In [7]:
data = Impute.impute_knn(data, 5)

/home/jupyter-khickey/.local/lib/python3.6/site-packages/impyute/util/preprocess.py:52: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  args[0] = args[0].as_matrix()


In [8]:
data.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,3.0,1.0,0.029074,51.345000,0.640426,0.054250,0.004875,0.009199,0.010929,1.0,3.864345,11.111111,1.0,0.0,0.0
1,4.0,0.0,-0.013352,25.936000,0.644753,0.051395,0.003258,0.011105,0.010876,0.0,12.028832,0.000000,1.0,0.0,0.0
2,2.0,1.0,0.020715,7.378000,0.636816,0.061764,0.011593,0.006271,0.011403,0.0,3.369134,0.000000,1.0,0.0,0.0
3,4.0,1.0,0.020023,8.526000,0.539634,0.061630,0.009686,0.007144,0.013319,0.0,3.299697,0.000000,1.0,0.0,0.0
4,1.0,1.0,-0.034895,632.298000,0.587413,0.048550,0.004518,0.010047,0.011261,1.0,3.726269,5.000000,0.0,0.0,1.0
5,2.0,1.0,-0.057556,197.591000,0.643137,0.045280,0.003445,0.005334,0.010056,1.0,5.724394,0.000000,0.0,0.0,1.0
6,3.0,1.0,0.004106,5.146000,0.765657,0.044295,0.005204,0.006727,0.008567,0.0,3.463095,0.000000,1.0,0.0,0.0
7,2.0,0.0,0.040620,506.948305,0.826347,0.053594,0.004899,0.010689,0.010868,0.0,2.421469,0.000000,1.0,0.0,0.0
8,2.0,1.0,-0.029316,279.600000,0.540323,0.061905,0.003915,0.008371,0.015797,0.0,4.816591,0.000000,0.0,0.0,1.0
9,2.0,1.0,-0.024925,494.008000,0.514286,0.050252,0.008868,0.009853,0.016094,1.0,3.056608,0.000000,1.0,0.0,0.0


In [9]:
dummies = pd.get_dummies(data[0], prefix = 'days_is_')
data = pd.concat([data, dummies], axis = 1)
data = data.drop(0, axis=1)

In [10]:
data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,days_is__1.0,days_is__2.0,days_is__3.0,days_is__4.0
0,1.0,0.029074,51.345,0.640426,0.054250,0.004875,0.009199,0.010929,1.0,3.864345,11.111111,1.0,0.0,0.0,0,0,1,0
1,0.0,-0.013352,25.936,0.644753,0.051395,0.003258,0.011105,0.010876,0.0,12.028832,0.000000,1.0,0.0,0.0,0,0,0,1
2,1.0,0.020715,7.378,0.636816,0.061764,0.011593,0.006271,0.011403,0.0,3.369134,0.000000,1.0,0.0,0.0,0,1,0,0
3,1.0,0.020023,8.526,0.539634,0.061630,0.009686,0.007144,0.013319,0.0,3.299697,0.000000,1.0,0.0,0.0,0,0,0,1
4,1.0,-0.034895,632.298,0.587413,0.048550,0.004518,0.010047,0.011261,1.0,3.726269,5.000000,0.0,0.0,1.0,1,0,0,0


In [16]:
data_copy = data.copy()
data_copy.columns

Index([             1,              2,              3,              4,
                    5,              6,              7,              8,
                    9,             10,             11,             12,
                   13,             14, 'days_is__1.0', 'days_is__2.0',
       'days_is__3.0', 'days_is__4.0'],
      dtype='object')

In [18]:
data_copy.drop(columns = ['1','12','13', '14', 'days_is__1.0', 'days_is__2.0', 'days_is__3.0', 'days_is__4.0'])
data_copy.head()
               
#Transform.skew_transform_log(data[:, 0:14])

KeyError: "['             1' '12' '13' '14'] not found in axis"

In [13]:
data.head(20)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,days_is__1.0,days_is__2.0,days_is__3.0,days_is__4.0
1,1.000000,-0.013352,3.255632,-0.438887,0.051395,-5.726618,0.011105,0.010876,-2.072327e+01,2.487306,-20.723266,2.718282,1.000000e-09,-2.072327e+01,-2.072327e+01,1.000000e-09,-2.072327e+01,1.000000e-09
2,2.718282,0.020715,1.998503,-0.451275,0.061764,-4.457393,0.006271,0.011403,-2.072327e+01,1.214656,-20.723266,2.718282,1.000000e-09,-2.072327e+01,-2.072327e+01,1.000000e+00,-2.072327e+01,-2.072327e+01
3,2.718282,0.020023,2.143120,-0.616864,0.061630,-4.637032,0.007144,0.013319,-2.072327e+01,1.193831,-20.723266,2.718282,1.000000e-09,-2.072327e+01,-2.072327e+01,1.000000e-09,-2.072327e+01,1.000000e-09
4,2.718282,-0.034895,6.449361,-0.532028,0.048550,-5.399715,0.010047,0.011261,1.000000e-09,1.315407,1.609438,1.000000,1.000000e-09,1.000000e-09,1.000000e-09,1.000000e-09,-2.072327e+01,-2.072327e+01
5,2.718282,-0.057556,5.286199,-0.441397,0.045280,-5.670937,0.005334,0.010056,1.000000e-09,1.744737,-20.723266,1.000000,1.000000e-09,1.000000e-09,-2.072327e+01,1.000000e+00,-2.072327e+01,-2.072327e+01
6,2.718282,0.004106,1.638220,-0.267022,0.044295,-5.258384,0.006727,0.008567,-2.072327e+01,1.242163,-20.723266,2.718282,1.000000e-09,-2.072327e+01,-2.072327e+01,1.000000e-09,1.000000e-09,-2.072327e+01
7,1.000000,0.040620,6.228409,-0.190740,0.053594,-5.318654,0.010689,0.010868,-2.072327e+01,0.884374,-20.723266,2.718282,1.000000e-09,-2.072327e+01,-2.072327e+01,1.000000e+00,-2.072327e+01,-2.072327e+01
8,2.718282,-0.029316,5.633360,-0.615589,0.061905,-5.542817,0.008371,0.015797,-2.072327e+01,1.572066,-20.723266,1.000000,1.000000e-09,1.000000e-09,-2.072327e+01,1.000000e+00,-2.072327e+01,-2.072327e+01
9,2.718282,-0.024925,6.202552,-0.664976,0.050252,-4.725310,0.009853,0.016094,1.000000e-09,1.117306,-20.723266,2.718282,1.000000e-09,-2.072327e+01,-2.072327e+01,1.000000e+00,-2.072327e+01,-2.072327e+01
10,2.718282,0.015487,-1.139434,-0.606136,0.066517,-5.296246,0.007084,0.012094,-2.072327e+01,0.934793,-20.723266,2.718282,1.000000e-09,-2.072327e+01,-2.072327e+01,1.000000e+00,-2.072327e+01,-2.072327e+01


In [35]:
data = Normalize.z_score(data)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.404995,0.654566,-0.407054,0.146508,0.310254,-0.105934,0.216231,-0.098612,1.046536,0.059313,1.269239,1.003035,-0.347193,-0.806226,-0.598352,-0.563352,1.753304,-0.577350
1,-2.469165,-0.619777,-0.753354,0.205661,-0.110057,-1.352194,0.817653,-0.119571,-0.955533,2.295653,-0.787897,1.003035,-0.347193,-0.806226,-0.598352,-0.563352,-0.570352,1.732051
2,0.404995,0.403480,-1.390815,0.097368,1.416642,2.573801,-1.007195,0.084493,-0.955533,-0.210771,-0.787897,1.003035,-0.347193,-0.806226,-0.598352,1.775090,-0.570352,-0.577350
3,0.404995,0.382698,-1.317483,-1.161107,1.396933,2.018136,-0.591262,0.754454,-0.955533,-0.251784,-0.787897,1.003035,-0.347193,-0.806226,-0.598352,-0.563352,-0.570352,1.732051
4,0.404995,-1.266863,0.866111,-0.557667,-0.528928,-0.341009,0.497966,0.030619,1.046536,-0.012345,1.198225,-0.996974,-0.347193,1.240347,1.671258,-0.563352,-0.570352,-0.577350


In [13]:
from scipy import stats

z = np.abs(stats.zscore(data))
thres = 3 # your defined threshold
print(np.where(z > thres))

(array([ 12,  22,  49,  57, 104, 105, 109, 157, 161, 165, 174, 184, 184,
       199, 243, 258, 263, 267, 279, 293, 303, 322, 371, 394, 402, 411,
       431, 440, 464, 519, 526, 541, 614, 646]), array([2, 2, 7, 9, 4, 1, 7, 9, 7, 9, 2, 4, 7, 9, 3, 9, 7, 6, 7, 9, 1, 2,
       7, 6, 7, 6, 2, 7, 3, 9, 4, 9, 5, 2]))


In [14]:
outliers = np.where(z > thres)
d = dict(zip(outliers[0], outliers[1]))
d

{12: 2,
 22: 2,
 49: 7,
 57: 9,
 104: 4,
 105: 1,
 109: 7,
 157: 9,
 161: 7,
 165: 9,
 174: 2,
 184: 7,
 199: 9,
 243: 3,
 258: 9,
 263: 7,
 267: 6,
 279: 7,
 293: 9,
 303: 1,
 322: 2,
 371: 7,
 394: 6,
 402: 7,
 411: 6,
 431: 2,
 440: 7,
 464: 3,
 519: 9,
 526: 4,
 541: 9,
 614: 5,
 646: 2}

In [ ]:
for key, value in d:
    data.loc[key, value]

In [12]:

for n in range(1, (data.shape[1] + 1)):
    print(n)
    # create a base classifier used to evaluate a subset of attributes
    from sklearn.linear_model import LogisticRegression
    from sklearn.feature_selection import RFE
    model = LogisticRegression()
    # create the RFE model and select 3 attributes
    rfe = RFE(model, n)
    rfe = rfe.fit(data, y1)
    # summarize the selection of the attributes
    print('Model with the best', n, 'features')
    print(dict(zip(data.columns, rfe.ranking_)))
    


1
Model with the best 1 features
{'C1': 15, 'C2': 5, 'C4': 6, 'C7': 14, '%_long_sentences': 2, '%_long_words': 8, '%_positive_words': 10, '%_negative_words': 9, '%_uncertain_words': 12, "C3'": 13, "C5'": 11, "C6'": 1, 'one_hot_Manufacturing': 3, 'one_hot_Other': 7, 'one_hot_Public Services': 4}
2
Model with the best 2 features
{'C1': 14, 'C2': 4, 'C4': 5, 'C7': 13, '%_long_sentences': 1, '%_long_words': 7, '%_positive_words': 9, '%_negative_words': 8, '%_uncertain_words': 11, "C3'": 12, "C5'": 10, "C6'": 1, 'one_hot_Manufacturing': 2, 'one_hot_Other': 6, 'one_hot_Public Services': 3}
3
Model with the best 3 features
{'C1': 13, 'C2': 3, 'C4': 4, 'C7': 12, '%_long_sentences': 1, '%_long_words': 6, '%_positive_words': 8, '%_negative_words': 7, '%_uncertain_words': 10, "C3'": 11, "C5'": 9, "C6'": 1, 'one_hot_Manufacturing': 1, 'one_hot_Other': 5, 'one_hot_Public Services': 2}
4
Model with the best 4 features
{'C1': 12, 'C2': 2, 'C4': 3, 'C7': 11, '%_long_sentences': 1, '%_long_words': 5, '

In [13]:
data.head()

,C1,C2,C4,C7,%_long_sentences,%_long_words,%_positive_words,%_negative_words,%_uncertain_words,C3',C5',C6',one_hot_Manufacturing,one_hot_Other,one_hot_Public Services
0,122.0,1.0,0.029074,51.345,0.640426,0.054250,0.004875,0.009199,0.010929,1,3.864345,11.111111,1,0,0
1,259.0,0.0,-0.013352,25.936,0.644753,0.051395,0.003258,0.011105,0.010876,0,12.028832,0.000000,1,0,0
2,90.0,1.0,0.020715,7.378,0.636816,0.061764,0.011593,0.006271,0.011403,0,3.369134,0.000000,1,0,0
3,209.0,1.0,0.020023,8.526,0.539634,0.061630,0.009686,0.007144,0.013319,0,3.299697,0.000000,1,0,0
4,80.0,1.0,-0.034895,632.298,0.587413,0.048550,0.004518,0.010047,0.011261,1,3.726269,5.000000,0,0,1
